In [1]:
import pandas as pd

In [ ]:
ls data

# ad.csv 크롤링 데이터
- `ad.csv`에서 `content_id` 기준으로 크롤링한 `ad_crawling.csv` 병합

In [2]:
ad_crawling = pd.read_csv('data/ad_new1.csv')
ad_df = pd.read_csv('data/ad.csv')

In [3]:
ad_df = pd.concat([ad_df, ad_crawling], axis=1)
ad_df = ad_df.drop('id', axis=1)
ad_df.head(2)

,content_id,user_id,name,keyword,price,flag_used,category_id_1,category_id_2,category_id_3,emergency_cnt,comment_cnt,interest,pfavcnt,b_pay,category,delivery_fee,place,status,text
0,157826057,114088,인텔 4세대 i5-4670 사무용 컴퓨터 팝니다.,"사무용컴퓨터,사무용PC,중고사무용컴퓨터,중고사무용PC,사무용중고컴퓨터",250000,1,600,600100,600100006,0,0,714,4,1,NaN,배송비포함,서울특별시 용산구 한강로동,판매중,"상품정보\n[ 컴퓨터 상세 사양 ] \n(SSD, 케이스만 신품, 나머지 중고)\n..."
1,162431268,3247363,갤럭시노트10+ 512GB 글로우 (무잔상) ! 0821,"갤럭시노트10,노트10플러스,노트10플러스중고,노트10플러스공기,노트10플러스글로",439000,1,600,600700,600700001,0,0,39,2,0,NaN,NaN,NaN,삭제됨,NaN


In [4]:
ad_crawling['status'].unique()

array(['판매중', '삭제됨', '거래 완료'], dtype=object)

- 저장

In [5]:
#ad_df.to_csv('data/ad_new.csv', index=False)

# 병합

In [116]:
impression_df = pd.read_csv('data/impression_log.csv')
view_df = pd.read_csv('data/view_log.csv')

#dataset_df = pd.read_csv('data/dataset.csv')

## impression + view
- label : 노출 - 0 , view - 1로 처리>\?

In [117]:
impression_df = impression_df.drop_duplicates()
view_df = view_df.drop_duplicates()

In [118]:
# impression 데이터에 log가 없는 view들의 id
no_imp_id = list(set(view_df['imp_id']) - set(impression_df['imp_id']))
len(no_imp_id)

460

In [120]:
# impression 데이터에 log가 없는 view들 모음
no_imp_view = view_df.set_index('imp_id').loc[no_imp_id].reset_index()

In [121]:
# 기존 view_df에서 impression과 겹치지 않는 id 제거
view_df = view_df.set_index('imp_id').drop(no_imp_id).reset_index()

In [122]:
view_df

,imp_id,server_time_kst,bid_price,device_type
0,99d1612dded122458e99,2021-08-31T16:56:46.013+09:00,50,a
1,998c612ddab9008cceee,2021-08-31T16:32:01.533+09:00,385,a
2,998c612dd77f007f2d67,2021-08-31T16:17:26.423+09:00,50,a
3,971a612ddff00eb50a47,2021-08-31T16:53:48.436+09:00,70,a
4,971a612ddd140ea93f30,2021-08-31T16:41:29.857+09:00,155,a
...,...,...,...,...
26478,9919612d14c1203c9bf0,2021-08-31T02:26:35.932+09:00,50,a
26479,97f9612d1250145c2e00,2021-08-31T02:16:06.295+09:00,60,a
26480,99d1612d19e220565606,2021-08-31T02:48:24.894+09:00,60,a
26481,99d1612d12bf2044dde5,2021-08-31T02:18:03.444+09:00,80,a


In [123]:
# impression_df의 imp_id, user_id 기준으로 view_df의 view들에 user_id 붙여줌
imp_user = impression_df[['imp_id', 'user_id', 'content_id']]
view_df = view_df.merge(imp_user, 'left').drop_duplicates()

### 라벨 생성

- 광고를 클릭했으면 1, 노출만 됐으면 0을 부여해서 병합

In [124]:
view_df['server_time_kst'] = view_df['server_time_kst'].apply(lambda x : x[11:-6])
impression_df['server_time_kst'] = impression_df['server_time_kst'].apply(lambda x : x[11:-6])

In [125]:
view_df['label'] = 1
view_df

,imp_id,server_time_kst,bid_price,device_type,user_id,content_id,label
0,99d1612dded122458e99,16:56:46.013,50,a,1728109,137078818,1
1,998c612ddab9008cceee,16:32:01.533,385,a,77328955,132419409,1
2,998c612dd77f007f2d67,16:17:26.423,50,a,75559871,162053601,1
3,971a612ddff00eb50a47,16:53:48.436,70,a,9461170,161384072,1
4,971a612ddd140ea93f30,16:41:29.857,155,a,10809468,162732366,1
...,...,...,...,...,...,...,...
26511,9919612d14c1203c9bf0,02:26:35.932,50,a,74774702,163161249,1
26512,97f9612d1250145c2e00,02:16:06.295,60,a,13008046,162188680,1
26513,99d1612d19e220565606,02:48:24.894,60,a,76011534,161297627,1
26514,99d1612d12bf2044dde5,02:18:03.444,80,a,1581224,142540474,1


In [126]:
impression_df['label'] = 0
impression_df

,imp_id,content_id,server_time_kst,bid_price,user_id,device_type,label
0,97cf612d3a7008353958,161592430,05:07:20.945,50,11555138,a,0
1,99d1612d39112085014f,162892746,05:02:38.277,50,1310312,a,0
2,97ca612d461111487741,162806628,05:57:03.384,55,76429349,a,0
3,97f9612d3995149e6b11,161593862,05:03:49.145,270,74800572,a,0
4,97f9612d3a22149eef25,161587624,05:06:26.494,50,4454345,a,0
...,...,...,...,...,...,...,...
852582,9919612e429a24183e0d,163247160,23:54:23.433,50,7660589,a,0
852583,998c612e425a027cdb47,78159183,23:54:17.029,65,3735841,a,0
852584,9919612e429724183042,152627901,23:54:24.948,70,9140358,a,0
852585,97f9612e4295183f6e39,160836652,23:54:20.398,165,1668136,a,0


In [129]:
# 같은 impression이 한 유저에게 여러번 나온 경우 한개만 사용
dup_imp_id = impression_df['imp_id'].value_counts()[impression_df['imp_id'].value_counts() > 1].index.tolist()
imp_drop_dup = impression_df.set_index('imp_id').loc[dup_imp_id].reset_index().groupby('imp_id').min().reset_index()
imp_no_dup = impression_df.set_index('imp_id').drop(dup_imp_id).reset_index()
impression_df = pd.concat([imp_no_dup, imp_drop_dup])

In [131]:
dup_view_id = view_df['imp_id'].value_counts()[view_df['imp_id'].value_counts() > 1].index.tolist()
view_drop_dup = view_df.set_index('imp_id').loc[dup_view_id].reset_index().groupby('imp_id').min().reset_index()
view_no_dup = view_df.set_index('imp_id').drop(dup_view_id).reset_index()
view_df = pd.concat([view_no_dup, view_drop_dup])

In [133]:
view_df

,imp_id,server_time_kst,bid_price,device_type,user_id,content_id,label
0,99d1612dded122458e99,16:56:46.013,50,a,1728109,137078818,1
1,998c612ddab9008cceee,16:32:01.533,385,a,77328955,132419409,1
2,998c612dd77f007f2d67,16:17:26.423,50,a,75559871,162053601,1
3,971a612ddff00eb50a47,16:53:48.436,70,a,9461170,161384072,1
4,971a612ddd140ea93f30,16:41:29.857,155,a,10809468,162732366,1
...,...,...,...,...,...,...,...
1425,99d1612e3e7e240f469b,23:37:12.512,50,a,11066993,163051037,1
1426,99d1612e3e9c240ff961,23:37:23.116,50,a,77215588,161121114,1
1427,99d1612e3fe02416f78f,23:42:44.411,50,a,75585834,161783213,1
1428,99d1612e40f1241cec6c,23:47:51.829,50,a,6113973,162520302,1


In [134]:
view_df['imp_time'] = impression_df.set_index('imp_id').loc[view_df['imp_id']]['server_time_kst'].tolist()
view_df = view_df.rename({'server_time_kst': 'view_time'}, axis=1)

In [135]:
view_df

,imp_id,view_time,bid_price,device_type,user_id,content_id,label,imp_time
0,99d1612dded122458e99,16:56:46.013,50,a,1728109,137078818,1,16:48:41.780
1,998c612ddab9008cceee,16:32:01.533,385,a,77328955,132419409,1,16:31:14.431
2,998c612dd77f007f2d67,16:17:26.423,50,a,75559871,162053601,1,16:17:21.879
3,971a612ddff00eb50a47,16:53:48.436,70,a,9461170,161384072,1,16:53:43.847
4,971a612ddd140ea93f30,16:41:29.857,155,a,10809468,162732366,1,16:41:10.127
...,...,...,...,...,...,...,...,...
1425,99d1612e3e7e240f469b,23:37:12.512,50,a,11066993,163051037,1,23:37:21.570
1426,99d1612e3e9c240ff961,23:37:23.116,50,a,77215588,161121114,1,23:37:36.715
1427,99d1612e3fe02416f78f,23:42:44.411,50,a,75585834,161783213,1,23:42:44.381
1428,99d1612e40f1241cec6c,23:47:51.829,50,a,6113973,162520302,1,23:47:47.267


In [136]:
impression_df = impression_df.set_index('imp_id').drop(view_df['imp_id']).reset_index()
impression_df = impression_df.rename({'server_time_kst': 'imp_time'}, axis=1)

In [137]:
impression_df

,imp_id,content_id,imp_time,bid_price,user_id,device_type,label
0,97cf612d3a7008353958,161592430,05:07:20.945,50,11555138,a,0
1,99d1612d39112085014f,162892746,05:02:38.277,50,1310312,a,0
2,97ca612d461111487741,162806628,05:57:03.384,55,76429349,a,0
3,97f9612d3995149e6b11,161593862,05:03:49.145,270,74800572,a,0
4,97f9612d3a22149eef25,161587624,05:06:26.494,50,4454345,a,0
...,...,...,...,...,...,...,...
826175,99d1612e363323df3c7c,146485304,23:01:40.005,75,2929116,a,0
826176,99d1612e3ac523f9aa89,163135587,23:20:56.553,60,2219768,a,0
826177,99d1612e3d022406dfe8,162418733,23:30:29.089,50,2269989,a,0
826178,99d1612e3f61241447a1,162346075,23:41:22.492,50,7230559,a,0


In [139]:
our_dataset = pd.concat([view_df, impression_df])

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


# 나머지 feature들 붙이기

In [144]:
ad_df = pd.read_csv('data/ad_new.csv')
advertiser_df = pd.read_csv('data/advertiser_title.csv')
viewer_df = pd.read_csv('data/viewer.csv')

### Ad.csv
- 크롤링한 데이터를 포함해서 컬럼명 조정 후 병합 진행

In [145]:
ad_dict = {'user_id': 'advertiser_id', 'name': 'content_name', 'keyword': 'content_keyword', 
           'price': 'content_price', 'flag_used': 'content_used', 
           'category_id_1': 'content_cat_1', 'category_id_2': 'content_cat_2', 'category_id_3': 'content_cat_3', 
          'emergency_cnt': 'content_emergency_count', 'comment_cnt': 'content_comment_count',
          'interest': 'content_views', 'pfavcnt': 'content_likes', 
          'status': 'content_status', 'b_pay': 'content_b_pay', 'place': 'content_place',
          'text': 'content_text', 'delivery_fee': 'content_delivery_fee'}
ad_df = ad_df.rename(ad_dict, axis=1)
ad_df.head(2)

,content_id,advertiser_id,content_name,content_keyword,content_price,content_used,content_cat_1,content_cat_2,content_cat_3,content_emergency_count,content_comment_count,content_views,content_likes,content_b_pay,category,content_delivery_fee,content_place,content_status,content_text
0,157826057,114088,인텔 4세대 i5-4670 사무용 컴퓨터 팝니다.,"사무용컴퓨터,사무용PC,중고사무용컴퓨터,중고사무용PC,사무용중고컴퓨터",250000,1,600,600100,600100006,0,0,714,4,1,NaN,배송비포함,서울특별시 용산구 한강로동,판매중,"상품정보\n[ 컴퓨터 상세 사양 ] \n(SSD, 케이스만 신품, 나머지 중고)\n..."
1,162431268,3247363,갤럭시노트10+ 512GB 글로우 (무잔상) ! 0821,"갤럭시노트10,노트10플러스,노트10플러스중고,노트10플러스공기,노트10플러스글로",439000,1,600,600700,600700001,0,0,39,2,0,NaN,NaN,NaN,삭제됨,NaN


- 일단 카테고리 대분류만 한글로 변경

In [146]:
cat2kor_1 = {220: '지역 서비스', 310: '여성 의류', 320: '남성 의류', 
             400: '남성 악세사리', 405: '신발', 410: '미용품', 420: '시계/쥬얼리', 430: '가방',
            500: '유아동/출산', 600: '전자제품', 700: '스포츠/레저', 750: '차량/오토바이', 800: '생활/가공식품',
            810: '가구/인테리어', 900: '도서/티켓/문구', 910: '스타굿즈', 920: '음반/악기', 930: '키덜트',
             980: '반려동물용품', 990: '예술/희귀/수집품',999: '기타'
            }
ad_df['content_cat_1'] = ad_df['content_cat_1'].apply(lambda x: cat2kor_1[x])
ad_df.head()

,content_id,advertiser_id,content_name,content_keyword,content_price,content_used,content_cat_1,content_cat_2,content_cat_3,content_emergency_count,content_comment_count,content_views,content_likes,content_b_pay,category,content_delivery_fee,content_place,content_status,content_text
0,157826057,114088,인텔 4세대 i5-4670 사무용 컴퓨터 팝니다.,"사무용컴퓨터,사무용PC,중고사무용컴퓨터,중고사무용PC,사무용중고컴퓨터",250000,1,전자제품,600100,600100006,0,0,714,4,1,NaN,배송비포함,서울특별시 용산구 한강로동,판매중,"상품정보\n[ 컴퓨터 상세 사양 ] \n(SSD, 케이스만 신품, 나머지 중고)\n..."
1,162431268,3247363,갤럭시노트10+ 512GB 글로우 (무잔상) ! 0821,"갤럭시노트10,노트10플러스,노트10플러스중고,노트10플러스공기,노트10플러스글로",439000,1,전자제품,600700,600700001,0,0,39,2,0,NaN,NaN,NaN,삭제됨,NaN
2,162806215,12851816,#한정판매#갤럭시S21 256GB 화이트 SS급 중고폰공기계[75333],"갤럭시S21,갤럭시S21화이트,S21화이트",620000,1,전자제품,600700,600700001,0,0,145,5,1,NaN,배송비포함,서울특별시 강남구 논현2동,판매중,"상품정보\n★ 번개장터 구매고객 특별이벤트 ★\n\n1,구입후 중고폰 100일 제품..."
3,152625380,2347538,#당일발송#갤럭시S9플러스 64G 대량판매 등급별초특가!,"S9플러스,S9플러스중고,갤럭시S9플러스",195000,1,전자제품,600700,600700001,0,0,234,8,0,NaN,NaN,NaN,삭제됨,NaN
4,161572473,12927760,포터2카고 중고화물차 팝니다.,"포터2,포터중고차,포터2카고,포터2중고,포터2카고중고",9800000,1,차량/오토바이,750200,750200700,0,0,388,4,0,NaN,NaN,NaN,삭제됨,NaN


- 병합

In [147]:
our_dataset = our_dataset.merge(ad_df, on='content_id')
our_dataset.head(2)

,bid_price,content_id,device_type,imp_id,imp_time,label,user_id,view_time,advertiser_id,content_name,...,content_emergency_count,content_comment_count,content_views,content_likes,content_b_pay,category,content_delivery_fee,content_place,content_status,content_text
0,50,137078818,a,99d1612dded122458e99,16:48:41.780,1,1728109,16:56:46.013,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,...,1,4,25583,817,0,NaN,NaN,NaN,삭제됨,NaN
1,50,137078818,a,99d1612d825321055790,10:14:09.313,1,10733469,10:14:16.926,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,...,1,4,25583,817,0,NaN,NaN,NaN,삭제됨,NaN


### Advertiser.csv

- 컬럼명 변경, 중복 제거

In [148]:
adv_name_dict = {'user_id': 'advertiser_id','comment_count': 'adv_comment_count', 'follower_count' : 'drop', 'pay_count': 'adv_pay_count', 
            'parcel_post_count': 'adv_parcel_post_count', 'transfer_count': 'adv_transfer_count', 'chat_count': 'adv_chat_count',
                'grade': 'adv_grade', 'favorite_count': 'adv_follower_count', 'review_count': 'adv_review_count', 'interest': 'adv_views',
                'title': 'adv_title', 'item_count': 'adv_item_count'}
advertiser_df = advertiser_df.rename(adv_name_dict, axis=1)
advertiser_df = advertiser_df.drop_duplicates()
advertiser_df

,advertiser_id,adv_follower_count,adv_grade,adv_item_count,adv_views,adv_review_count,adv_comment_count,drop,adv_pay_count,adv_parcel_post_count,adv_transfer_count,adv_chat_count,adv_title
0,2487730,1899,7673,431,31681,775,529,0,575,0,35,130,이수컴
1,1462217,799,5520,433,16794,570,250,0,942,0,57,11,최저가전
2,6972746,3692,7526,2530,36944,776,828,0,325,0,10,13,북부통신
3,1358510,1228,2106,100,27142,223,431,0,232,9,1,4,바이크빌드
5,5051200,254,692,77,6462,74,0,0,196,8,40,0,스마일Tool
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,75231266,28,38,193,69,5,0,0,11,1,0,2,그림겔러리정리
1513,5332936,2,51,12,289,5,1,0,1,0,0,0,현규대장
1514,8173665,4,10,16,26,1,5,0,2,0,0,0,아리아리12
1524,2294187,462,3720,833,1228,376,0,0,186,1,6,60,저렴이중고가게


- 병합

In [149]:
our_dataset = our_dataset.merge(advertiser_df, on='advertiser_id')
our_dataset.head(2)

,bid_price,content_id,device_type,imp_id,imp_time,label,user_id,view_time,advertiser_id,content_name,...,adv_item_count,adv_views,adv_review_count,adv_comment_count,drop,adv_pay_count,adv_parcel_post_count,adv_transfer_count,adv_chat_count,adv_title
0,50,137078818,a,99d1612dded122458e99,16:48:41.780,1,1728109,16:56:46.013,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,...,247,4525,542,739,0,496,0,0,11,AIROK
1,50,137078818,a,99d1612d825321055790,10:14:09.313,1,10733469,10:14:16.926,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,...,247,4525,542,739,0,496,0,0,11,AIROK


### Viewer.csv

- 컬럼명 변경 및 중복 제거

In [150]:
viewer_name_dict = {'comment_count': 'viewer_comment_count', 'following_cnt' : 'viewer_following_count', 'pay_count': 'viewer_pay_count', 
            'parcel_post_count': 'viewer_parcel_post_count', 'transfer_count': 'viewer_transfer_count', 'chat_count': 'viewer_chat_count',
                   'gender': 'viewer_gender', 'age': 'viewer_age'}
viewer_df = viewer_df.rename(viewer_name_dict, axis=1)
viewer_df = viewer_df.drop_duplicates()
viewer_df

,user_id,viewer_gender,viewer_age,viewer_following_count,viewer_pay_count,viewer_parcel_post_count,viewer_transfer_count,viewer_chat_count
0,513808,2,48,0,0,0,0,0
1,10764680,1,25,0,0,0,0,0
2,516424,0,0,1,0,0,0,0
3,10248440,1,36,14,0,0,0,0
4,9741736,1,30,2,0,0,0,0
...,...,...,...,...,...,...,...,...
56376,77099027,2,60,1,0,0,0,1
56377,77172013,1,47,17,0,0,0,0
56378,77220633,2,51,4,0,0,0,0
56379,77270759,1,51,0,0,0,0,0


- 병합

In [151]:
our_dataset = our_dataset.merge(viewer_df, on='user_id')
our_dataset.head(2)

,bid_price,content_id,device_type,imp_id,imp_time,label,user_id,view_time,advertiser_id,content_name,...,adv_transfer_count,adv_chat_count,adv_title,viewer_gender,viewer_age,viewer_following_count,viewer_pay_count,viewer_parcel_post_count,viewer_transfer_count,viewer_chat_count
0,50,137078818,a,99d1612dded122458e99,16:48:41.780,1,1728109,16:56:46.013,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,...,0,11,AIROK,1,30,5,0,0,0,0
1,50,137078818,a,971a612de1a50ebc01ce,17:00:45.805,0,1728109,NaN,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,...,0,11,AIROK,1,30,5,0,0,0,0


# 데이터 정리 후 저장

- 필요 없는 컬럼 제거

In [152]:
our_dataset = our_dataset.drop('drop', axis=1)

In [153]:
our_dataset = our_dataset.drop('device_type', axis=1)

In [154]:
our_dataset = our_dataset.rename({'advertiser_id': 'adv_id'}, axis=1)

In [155]:
our_dataset = our_dataset.drop('category', axis=1)

- 이미지 url 생성

In [156]:
our_dataset['content_img_url'] = our_dataset['content_id'].apply(lambda x: f'https://media.bunjang.co.kr/product/{x}_...')

In [157]:
our_dataset.iloc[1]

bid_price                                                                  50
content_id                                                          137078818
imp_id                                                   971a612de1a50ebc01ce
imp_time                                                         17:00:45.805
label                                                                       0
user_id                                                               1728109
view_time                                                                 NaN
adv_id                                                               12851816
content_name                               #당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!
content_keyword                           아이폰SE2중고,아이폰se2공기계,SE2중고,SE2,아이폰SE2
content_price                                                          250000
content_used                                                                1
content_cat_1                                                   

In [160]:
drop_name_list = []
for c_name in ad_df['content_name']:
    if '매입' in c_name:
        print(c_name)
        drop_name_list.append(c_name)
    elif '삽니다' in c_name:
        print(c_name)
        drop_name_list.append(c_name)

고장 파손 아이폰 등 중고폰 액정깨진 폰 삽니다.
갤럭시 버즈2 미개봉 무한매입 합니다
명품매입 선입금 출장매입 최고가 루이비통 샤넬 에르메스 구찌 스톤아일랜드
[초고가]90%구글기프트카드, 구글기프트코드, 구글교환권, 구글핀 매입
중고폰매입 아이폰12.아이폰11.아이폰XS.아이폰XR.아이폰X.아이폰8
파손폰매입 아이폰12프로 아이폰12프로맥스 노트8 노트9 노트10
기프티콘 매입해요 :)
중고폰매입 파손폰 중고폰삽니다 아이폰11프로 아이폰xs 고장폰 아이폰12
@기프티콘 최고가 매입성지@  모든 기프트콘 상품권 최고가 매입
중고폰매입 아이폰xs 아이폰xr 아이폰xsmax 아이폰se2 아이폰11
중고폰매입 아이폰7 아이폰7플러스 아이폰8 아이폰8플러스 아이폰x
중고폰매입 고장폰 중고폰삽니다 아이폰12프로 파손폰 아이폰11 아이폰x
명품매입/지방시/루이비통/구찌/입생로랑/디올/오프화이트/스톤/톰브라운
삼) 아이폰 .고장 파손 모든 휴대폰 삽니다.
박스폰 수도권 최고가 매입!
24시 컬쳐랜드(문화상품권) 해피머니 북앤라이프 상품권 삽니다.
문상91%<삽니다>컬쳐랜드91해피머니 문상 현금교환 문화상품권현금화
중고폰매입 아이폰11프로 아이폰11프로맥스 아이폰12미니 아이폰12
중고폰매입 s10플러스 s105g s20 s20플러스 s20울트라
중고폰매입 파손폰 중고폰삽니다 아이폰11프로 고장폰 아이폰12 아이폰xr
중고명품 매입합니다
명품매입 당일입금처리 최고가매입 당일현금지급
중고폰매입 노트10플러스 노트20 노트20울트라 s8 s9 s10


In [165]:
drop_name_id = ad_df.set_index('content_name').loc[drop_name_list]['content_id'].tolist()

In [169]:
our_dataset = our_dataset.set_index('content_id').drop(drop_name_id)

In [173]:
our_dataset['content_used'] = our_dataset['content_used'].apply(lambda x : x % 2)

- 용량 문제로 full/text제외 나눠서 저장

In [175]:
our_notext = our_dataset.drop(['content_keyword', 'content_text', 'adv_title'], axis=1)

In [176]:
our_dataset.to_csv('data/merged_data_full.csv', index=False)
our_notext.to_csv('data/merged_data_notext.csv', index=False)

In [29]:
our_dataset = pd.read_csv('data/our_data_full.csv')

## 노션 업로드용 데이터 정리 후 저장

In [24]:
our_dataset[our_dataset['content_status']!='삭제됨'].sort_values('imp_id').iloc[3:13][['imp_id', 'content_id', 'user_id', 'adv_id', 'label', 'bid_price', 'server_time_kst',
       'content_name', 'content_keyword', 'content_price',
       'content_used', 'content_cat_1', 'content_cat_2', 'content_cat_3',
       'content_emergency_count', 'content_comment_count', 'content_views',
       'content_likes', 'content_b_pay', 'content_delivery_fee',
       'content_place', 'content_status', 'content_text', 'content_img_url',
        'adv_follower_count', 'adv_grade', 'adv_item_count', 'adv_views', 'adv_review_count',
       'adv_comment_count', 'adv_pay_count', 'adv_parcel_post_count',
       'adv_transfer_count', 'adv_chat_count', 'adv_title', 'viewer_gender',
       'viewer_age', 'viewer_following_count', 'viewer_pay_count',
       'viewer_parcel_post_count', 'viewer_transfer_count',
       'viewer_chat_count']].to_csv('노션용.csv', index=False)

In [25]:
notion = pd.read_csv('노션용.csv')

In [31]:
our_dataset.loc[2]#[['content_name', 'content_keyword', 'content_text', 'content_img_url', 'adv_title']]

bid_price                                                                  50
content_id                                                          162892746
imp_id                                                   971a612d3e480cf3d15a
label                                                                       0
server_time_kst                                 2021-08-31T05:23:48.340+09:00
user_id                                                               1310312
adv_id                                                               12851816
content_name                           #한정판매#갤럭시Z플립2 그레이 256GB 중고폰 공기계[26583]
content_keyword                                          갤럭시Z플립2,Z플립2그레이,Z플립2
content_price                                                          620000
content_used                                                                1
content_cat_1                                                            전자제품
content_cat_2                                                   